In [ ]:
#!pip install cassandra-driver
#version python 3.7

In [ ]:
#Conexión a la cassandra que corre en mi docker
from cassandra.cluster import Cluster

#cluster = Cluster(contact_points=['192.168.99.100'],port=9042)
cluster = Cluster(contact_points=['localhost:9091'],port=9042)
session=cluster.connect()

In [ ]:
#Una visión general del fichero que quiero insertar en cassandra
import pandas as pd
import numpy as np

crimen=pd.read_csv('./crime_rate_Spain.csv',sep=',',header=0)
crimen.head()

In [ ]:
crimen.columns

In [ ]:
crimen.dtypes

In [ ]:
#Creo el keyspace y la tabla

session.execute(
    """CREATE KEYSPACE IF NOT EXISTS PROYECTO WITH replication = {
        'class':'simpleStrategy',
        'replication_factor':1 }; """
)

In [ ]:
session.set_keyspace("proyecto")

In [ ]:
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import models
from cassandra.cqlengine import connection

class Crimen(Model):
    Location = columns.Text(primary_key=True)
    year = columns.Integer(primary_key=True)
    crime = columns.Text()
    total_cases = columns.Integer()

In [ ]:
connection.register_connection('cluster', session=session)
from cassandra.cqlengine.management import sync_table
sync_table(Crimen,keyspaces=['proyecto'],connections=['cluster'])

In [ ]:
for row in session.execute("""select column_name, kind, type from system_schema.columns WHERE keyspace_name = 'proyecto' ;"""):
    print(row)

In [ ]:
#Inserto los datos en mi tabla creada


for index, valeu in crimen.interrows():
    session.execute("""INSERT INTO proyecto.crimen(Location,year,crime,total_cases))
    VALUES(%s,%s,%s,%s)""",
    (value['Location'], value['year'], value['crime'], value['total_cases']))

In [ ]:
#consulta en formato JSON:

all_rows=session.execute("""SELECT * FROM proyecto.crimen""")

for row in all_rows[:5]:
    print(row)

In [ ]:
#consulta en formato datframe:
df=pd.DataFrame()
for row in all_rows[:5]:
    df=df.append(row,ignore_index=True)

In [ ]:
df.head()